In [1]:
import os
from qiskit import execute
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit

from quantuminspire.credentials import get_authentication
from quantuminspire.qiskit import QI

QI_URL = os.getenv('API_URL', 'https://api.quantum-inspire.com/')




<a id="layout"></a>
# 1. Quantum Key Distribution Activity layout

In project, we are going to implement the E91 protocol. Steps of the protocol:

1. Create entangled states
2. Alice randomly selects a basis ( × or + )
3. Bob randomly selects a basis ( × or + )
4. `Intermediate interface function 1`: a MUX that creates a circuit based on Alice and Bob's selection of bases
5. `Intermediate interface function 2`: execute the measurement on Quantum Inspire 
6. `Intermediate interface function 3`: send back measurement results to Alice and Bob
7. `ANNOUNCE BASIS`: Alice announces which basis she used to encode each bit, via the classical channel.

7. `FIND SYMMETRIC KEY`: Alice and Bob discard bits in their key that used a different basis.

These 7 steps allow a key to be distributed between Alice and Bob securely, now the two can send secure and encrypted messages through an insecure channel. In the required content for this lab, we will implement steps 1-4, and in the optional content we will implement steps 5-7.

Note that we have left out **Step8: Analysis** here. In this lab, we will not worry about an eavesdropper, but focus on the code for the basic protocol. Therefore, Alice and Bob don't need to run an analysis step. If you're interested, you can try implementing code for Eve in the optional challenge section at the end of the notebook!

In [2]:
from random import getrandbits
import qiskit as q

In [4]:
# Parameters
N_en_pairs = 4
Alice_seq = []
Bob_seq = []

<a id="layout"></a>
## 1. Create entangled states

In [16]:
def measurement(i):
    if alice_seq[i]== "1":
        qc.measure(Alice_Reg,cr[0])
    elif alice_seq[i] == "2":
        qc.h(Alice_Reg)
        qc.measure(Alice_Reg,cr[0])
    elif alice_seq=="2":
        qc.s(Alice_Reg)
        qc.h(Alice_Reg)
        qc.tdag(Alice_Reg)
        qc.h(Alice_Reg)
        qc.measure(Alice_Reg, cr[0])
        
    if bob_seq[i]=="1":
        qc.s(Bob_Reg)
        qc.h(Bob_Reg)
        qc.t(Bob_Reg)
        qc.h(Bob_Reg)
        qc.measure(Bob_Reg, cr[1])
    elif bob_seq[i] == "2":
        qc.s(Bob_Reg)
        qc.h(Bob_Reg)
        qc.tdag(Bob_Reg)
        qc.h(Bob_Reg)
        qc.measure(Bob_Reg, cr[1])
    elif bob_seq[i] == "3":
        qc.h(Bob_Reg)
        qc.measure(Bob_Reg, cr[1])

In [9]:
Quantum_Circuit = []

for i in range(N_en_pairs):
    Alice_Reg = QuantumRegister(1, name="alice")
    Bob_Reg = QuantumRegister(1, name="bob")
    cr = ClassicalRegister(4, name="cr")
    qc = QuantumCircuit(Alice_Reg, Bob_Reg, cr)
    
    #Create Entanglement
    
    qc.h(Alice_Reg)
    qc.cx(Alice_Reg, Bob_Reg)
    
    #Measurement Curcuit
    measurement(i)
    
    Quantum_Circuit.append(qc)
    #for i in range(N_en_pairs):
    #qc.h(Alice_Reg[i])
    #qc.cx(Alice_Reg[i], Bob_Reg[i])
#Quantum_Circuit.draw(output='mpl')
Quantum_Circuit

In [30]:
def Al_Bob_rand_basis(length):
    
    # This stores the bases that Alice will prepare the states in
    alice_bases = ""
    bob_bases = ""
    
    # For the length 
    for i in range(length):
        # We use the function getrandbits to get either a 0 or 1 randomly,
        # 0 means encode in the (0,1) basis and 1 means encode in the (+,-) basis
        alice_bases += (str(getrandbits(1)))
        bob_bases += (str(getrandbits(1)))
    
    # return the string of bits and the list of bases they should be encoded in
    return alice_bases, bob_bases

In [32]:
alice_bases, bob = Al_Bob_rand_basis(N_en_pairs)

In [33]:
alice

'0101'

In [13]:
def testing():
    from quantuminspire.credentials import save_account
    save_account('35ef5a0ed17d10ac317893831724a209e47815a5')
    project_name = 'E91_test'
    authentication = get_authentication()
    QI.set_authentication(authentication, QI_URL, project_name=project_name)
    # Create an interface between Qiskit and Quantum Inpsire to execute the circuit
    qi_backend = QI.get_backend('QX single-node simulator')
    job = execute(Quantum_Circuit, qi_backend)

In [18]:
job.get_counts(Quantum_Circuit)

AttributeError: 'QIJob' object has no attribute 'get_counts'